In [59]:
import numpy as np 
import numpy.linalg as npla

In [60]:
def LUfactor(A, pivoting = True):
    """Factor a square matrix with partial pivoting, A[p,:] == L @ U
    Parameters: 
      A: the matrix.
      pivoting: whether or not to do partial pivoting
    Outputs (in order):
      L: the lower triangular factor, same dimensions as A, with ones on the diagonal
      U: the upper triangular factor, same dimensions as A
      p: the permutation vector that permutes the rows of A by partial pivoting
    """
    
    # Check the input
    m, n = A.shape
    assert m == n, 'input matrix A must be square'
    
    # Initialize p to be the identity permutation
    p = np.array(range(n))
    
    # Make a copy of the matrix that we will transform into L and U
    LU = A.astype(np.float64).copy()
    
    # Eliminate each column in turn
    for piv_col in range(n):
     
        # Choose the pivot row and swap it into place
        if pivoting:
            piv_row = piv_col + np.argmax(np.abs(LU[piv_col:, piv_col]))   # Added np.abs() to fix bug
            assert LU[piv_row, piv_col] != 0., "can't find nonzero pivot, matrix is singular"
            # print("Before:")
            # print(LU)

            LU[[piv_col, piv_row], :]  = LU[[piv_row, piv_col], :]
            p[ [piv_col, piv_row] ]      = p[[piv_row, piv_col]]
            
            # print("After: ")
            # print(LU)
        # Update the rest of the matrix
        pivot = LU[piv_col, piv_col]
        assert pivot != 0., "pivot is zero, can't continue"
        
        # This is the standard "core" of the algorithm (same as in LUfactorNoPiv)
        for row in range(piv_col + 1, n):
            multiplier = LU[row, piv_col] / pivot
            LU[row, piv_col] = multiplier
            LU[row, (piv_col+1):] -= multiplier * LU[piv_col, (piv_col+1):]
            # print("Did add on row: " + str(row) + "Column: " + str(piv_col))
            
    # Separate L and U in the result
    U = np.triu(LU)
    L = LU - U + np.eye(n)
    
    # This will return the L, U, AS WELL AS p (the permutation vector).
    return (L, U, p)

In [61]:
def Lsolve(L, b):
    """Forward solve a unit lower triangular system Ly = b for y
    Parameters: 
      L: the matrix, must be square, lower triangular, with ones on the diagonal
      b: the right-hand side vector
    Output:
      y: the solution vector to L @ y == b
    """
    
    # Check the input
    m, n = L.shape
    assert m == n, "matrix L must be square"
    assert np.all(np.tril(L) == L), "matrix L must be lower triangular"
    assert np.all(np.diag(L) == 1), "matrix L must have ones on the diagonal"
    
    # Make a copy of b that we will transform into the solution
    y = b.astype(np.float64).copy()
    
    # Forward solve
    for col in range(n):
        y[col+1:] -= y[col] * L[col+1:, col]
        
    return y

In [62]:
def Usolve(U, y):
    # Check the input
    m, n = U.shape
    assert m == n, "matrix U must be square"
    assert np.all(np.triu(U) == U), "matrix U must be upper triangular"
    assert np.all(np.diag(U) != 0), "matrix U must have nonzeros on the diagonal"
    # Make a copy of y that we will transform into the solution
    x = y.astype(np.float64).copy()
    # Backward solve
    for row in reversed(range(n)):
        x[row] /= U[row, row]
        x[:row] -= x[row] * U[:row, row]
        
    return x


In [79]:
import numpy as np
import numpy.linalg as npla
t = 10**-20
A = np.array([[t,1],[1,1]])
b = np.array([t+2,3])
L, U, p = LUfactor(A, True)
y = Lsolve(L, b[p])
x = Usolve(U, y)
t_predict = np.array([[1,2]])
print("\nresidual norm:", npla.norm(t_predict - x))
L, U, p = LUfactor(A, False)
y = Lsolve(L, b[p])
x = Usolve(U, y)
t_predict = np.array([[1,2]])
print(x)
print("\nresidual norm:", npla.norm(t_predict - x))


residual norm: 0.0
[0. 2.]

residual norm: 1.0


In [64]:
import numpy as np

A = np.random.randint(0, 10, size=(5, 5))
b = np.array([1,2,3,4,5])
L, U, p = LUfactor(A, True)
y = Lsolve(L, b[p])
print("U: \n", U)
x = Usolve(U, y)

U: 
 [[ 8.          1.          8.          4.          3.        ]
 [ 0.          6.375       1.         -1.5         0.125     ]
 [ 0.          0.          8.21568627 10.17647059 -0.09803922]
 [ 0.          0.          0.         10.24582339  7.83770883]
 [ 0.          0.          0.          0.         -2.69718146]]
